# 전처리

In [1]:
import numpy as np
import csv
#csv는 엑셀과 달리 , 를 통해 데이터를 구분

In [2]:
#given cities
cities = []
#solution
sol = []

# Euclidean distance measuring function

In [3]:
def distance(x,y):
    dist = np.linalg.norm(np.array(x)- np.array(y))
    return dist 
# np.linalg.norm -> L2 norm 벡터 p,q의 직선 거리 
# (x^2 + y^2)^(1/2)

# 데이터를 가져오기 및 확인

In [4]:
with open('example_solution.csv', mode= 'r', newline = '') as solution:
    
    # mode r -> 읽기 모드이다
    #csv의 행단위로 데이터를 출력
    #각 line은 ㅣist형식으로 되어있다
    #newline = '' 데이터가 띄어쓰기 없이 들어간다

    #read solution sequence -> 즉 example_solution을 읽어드림
    reader = csv.reader(solution)
    for row in reader:
        #print(row[0]) #0행의 숫자들
        sol.append(int(row[0]))
        if(len(sol) < 10):
            print(sol[0:5])
    print('#########################################################################')
    print(sol[0:15])

[760]
[760, 21]
[760, 21, 950]
[760, 21, 950, 122]
[760, 21, 950, 122, 807]
[760, 21, 950, 122, 807]
[760, 21, 950, 122, 807]
[760, 21, 950, 122, 807]
[760, 21, 950, 122, 807]
#########################################################################
[760, 21, 950, 122, 807, 4, 277, 423, 688, 241, 494, 605, 499, 527, 656]


# 1. get solution sequence and reorfering (sort from 0)

In [5]:
with open('example_solution.csv', mode= 'r', newline = '') as solution:
    
    # mode r -> 읽기 모드이다
    #csv의 행단위로 데이터를 출력
    #각 line은 ㅣist형식으로 되어있다
    #newline = '' 데이터가 띄어쓰기 없이 들어간다

    #read solution sequence -> 즉 example_solution을 읽어드림
    reader = csv.reader(solution)
    for row in reader:
        sol.append(int(row[0]))
    print('The index of 0 is :',sol.index(0))    
    #reordering solution sequence
    idx = sol.index(0) # 0이라는 숫자의 index
    
    front = sol[idx:]
    back = sol[0:idx]
    
    print('The front 0~20 is : ',front[0:20])
    print('/////////////////////////////////////////////////////////////////////////////////////////////////////////////')
    print('The back 0~20 is : ',back[0: 20])
    sol = front + back
    
    #expand 0 city (start) for simplicity
    sol.append(int(0))

The index of 0 is : 189
The front 0~20 is :  [0, 934, 200, 370, 524, 262, 982, 902, 71, 806, 744, 958, 430, 153, 81, 140, 786, 837, 634, 259]
/////////////////////////////////////////////////////////////////////////////////////////////////////////////
The back 0~20 is :  [760, 21, 950, 122, 807, 4, 277, 423, 688, 241, 494, 605, 499, 527, 656, 29, 872, 753, 165, 948]


즉 0을 기준으로 앞과 뒤를 짤라서 0 부터 시작하게 끔 자르고 마지막은 0에 도달 해야 하니깐 0을 추가                            
ex) 1 2 3 0 4 5 6             
front = 0 4 5 6           
back = 1 2 3           
sol = 0 4 5 6 1 2 3         

sol.append(int(0)) 
->최종 결과 0 4 5 6 1 2 3 0

위의 예시에서는         
0 의 index 값이 189 이므로     
front 에 189 ~ 999 index에 해당하는 숫자가 들어가고        
back 에 0 ~ 188 index에 해당하는 숫자가 들어간다

# 2. get TSP city map

In [6]:
with open('TSP.csv', mode = 'r', newline='') as tsp:
    #마찬가지로 읽기 방식으로 TSP.csv를 가져온다 -> 0행에는 x좌표 1행에는 y 좌표가 들어 있음
    
    #read TSP city map
    reader = csv.reader(tsp)
    for row in reader: # -> 가로 축으로 읽으므로 각 열 별로[x,y] 형태로 들어가게 된다
        cities.append(row)
print('cities 에 들어간 원소의 개수 :',len(cities))
print(cities[:25])

cities 에 들어간 원소의 개수 : 1000
[['53.25009344431329', '22.444885209100597'], ['21.570928175048255', '17.44707561216037'], ['72.16107675668404', '85.88179688723211'], ['52.59628100814437', '68.0997652916171'], ['99.55503411642916', '89.03868014012214'], ['45.263493711035', '24.433107718919345'], ['51.65884282004931', '90.10642634366273'], ['89.7286579150431', '69.21655768344208'], ['39.00003159887097', '80.55196125207841'], ['35.59526184197473', '14.05610647225216'], ['66.19318558706838', '55.68426760836053'], ['34.1551559598477', '85.84359753107817'], ['37.75840849411198', '8.948084373549136'], ['65.16521495804488', '93.65696789745532'], ['42.69022506258199', '92.09116283653597'], ['34.978245803048665', '90.63114224147267'], ['31.624449741999662', '37.20161513918061'], ['64.27557457661524', '47.52318749669915'], ['83.64758869867107', '0.8456953312116822'], ['66.90344399989151', '50.71682553053572'], ['16.884423905642787', '87.22366700571314'], ['87.5480501416314', '69.49436954110595'], ['5

# 3. evaluate solution cost

In [7]:
total_cost = 0

for idx in range(len(sol)-1): #0~ 999 까지
    
    #get city positions
    pos_city_1 = [float(cities[sol[idx]][0]), float(cities[sol[idx]][1])] #현재 node의 좌표를 float형으로 가져옴
    pos_city_2 = [float(cities[sol[idx+1]][0]), float(cities[sol[idx+1]][1])] #다음 node의 좌표를 float형으로 가져옴
    
    #distance calculation
    dist = distance(pos_city_1,pos_city_2) # 두 node 사이 의 거리를 구하여서
    
    #accumulation
    total_cost += dist # 토탈에 추가
    
print('final cost: ' +str(total_cost))

final cost: 102629.4310080144
